# LLM Prototyping Notebook

Quick experiments with LLM providers

In [1]:
# Load environment and imports
import os
from dotenv import load_dotenv
load_dotenv()

import sys
sys.path.append('.')

from modules.llm_provider import agent
from pydantic import BaseModel, Field
from typing import Literal, List, Optional, Union
from enum import Enum
import json
from time import time
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# ⚠️ Clear All Outputs before committing to prevent leaking sensitive data! ⚠️

# Changelog

All notable changes to **Screening Cheatsheet Response Schema** will be documented in this cell.

The format is based on [Keep a Changelog](https://keepachangelog.com/en/1.1.0/),
and this project adheres to [Semantic Versioning](https://semver.org/spec/v2.0.0.html).

## [Unreleased]
### Added
- Example cheat sheets templates, inputs, and outputs
### Changed
- Latest changelog revisions
- Updated `.env.example` to follow latest changes
- Response options are dynamically detected from input
## [0.0.1-alpha.2+8404e0a] - 2025-06-17
### Added
- `FinalDecision`
- Default flow diagram pseudocode in `ScreeningCheatsheet`
- `is_last_step` in `FlowDiagramStep` for pseudocode to make sense
- Some [provisions](https://github.com/pvzhelnov/cheatsheet_parser/commit/7c6fae296de066c761db7a4dd7a224eebf64ae7f) against leaking sensitive data
- This changelog
### Changed
- Definitions in `FlowDiagramStep` to handle final decision
- Handling of flow diagram in `ScreeningCheatsheet` to accommodate additions
- Migrated the repo to a [more recent version](https://github.com/pvzhelnov/gerpa/tree/1ff0f1bcd1c63e4dd27a3f0b4e052f0bbad70bb6) of GERPA
- Quite importantly, the migration to new GERPA led to some changes to default LLM inference settings, in particular, all agents are now run with the following config (unless requested otherwise):
  - `temperature` = 0.0
  - `top_k` = 40
  - `top_p` = 0.95
  - `seed` = 42 (if offered by provider)
  - `safety_settings` set to provider-specific values to turn all safety provisions off
### Tested
- On the same sample cheat sheet
## 0.0.1-alpha.1+outside.of.this.repo - 2025-06-03
### Added
- `ScreeningCheatsheet` and all the scaffolding
- System instruction (SHA-256 hash: `64362907`)
- Model used: `gemini-2.5-flash-preview-05-20`
### Tested
- On a sample cheat sheet

[unreleased]: https://github.com/pvzhelnov/cheatsheet_parser/compare/8404e0a0c6332addeb2992cfda5d1ed3f68a2469...HEAD
[0.0.1-alpha.2+8404e0a]: https://github.com/pvzhelnov/cheatsheet_parser/tree/8404e0a0c6332addeb2992cfda5d1ed3f68a2469

In [2]:
class CheatsheetQuestionUID(BaseModel):
    unique_question_id: int

# TO DO: Separate into two LLM calls and implement using generics

class ResponseOptionUniqueLiteral(BaseModel):
    string_value: str = Field(..., description="Capture the literal value verbatim as described in the input file(s).")
    unique_literal: Literal['Yes', 'No', 'Maybe', 'Other'] = Field(..., description="May occasionally include other literal value types depending on context. Select the closest one in case of obvious variants, or select Other if no matching literal value is found.")

class ResponseOption(BaseModel):
    value: ResponseOptionUniqueLiteral
    notes: List[str] = Field(..., description="if the study...")

class CheatsheetQuestion(BaseModel):
    question_uid: CheatsheetQuestionUID
    question_formulation: str
    responses: List[ResponseOption]
    question_note: Optional[str] = Field(..., description="Any question-wide note(s), if present.")

class FinalDecision(BaseModel):
    decision: Literal['Exclude', 'Include']

class OnResponseOption(BaseModel):
    response_option_unique_literal: ResponseOptionUniqueLiteral
    action_to_take: Union[CheatsheetQuestionUID, FinalDecision]

class FlowDiagramStep(BaseModel):
    step_id: int
    is_last_step: bool
    on_response_option: List[OnResponseOption]

class ScreeningCheatsheet(BaseModel):
    allowed_response_options: List[ResponseOptionUniqueLiteral] = Field(..., description="Deduce these from the entire preceding context. Consider obviously equivalent response options like case differences or typos to be equivalent and select the most consistent variant.")
    questions: List[CheatsheetQuestion]
    flow_diagram_detected: bool = Field(..., description="Is there a flow diagram in the input file(s)?")
    flow_diagram: List[FlowDiagramStep] = Field(..., description="If flow_diagram_detected, extract it. Otherwise implement the following pseudocode: { While is_last_step is False: { If Maybe/Yes Then Proceed to next step_id, Else Exclude } ; Once is_last_step is True: { If Maybe/Yes Then Include, Else Exclude } }")

print(json.dumps(ScreeningCheatsheet.model_json_schema(), indent=2, ensure_ascii=False))

{
  "$defs": {
    "CheatsheetQuestion": {
      "properties": {
        "question_uid": {
          "$ref": "#/$defs/CheatsheetQuestionUID"
        },
        "question_formulation": {
          "title": "Question Formulation",
          "type": "string"
        },
        "responses": {
          "items": {
            "$ref": "#/$defs/ResponseOption"
          },
          "title": "Responses",
          "type": "array"
        },
        "question_note": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          ],
          "description": "Any question-wide note(s), if present.",
          "title": "Question Note"
        }
      },
      "required": [
        "question_uid",
        "question_formulation",
        "responses",
        "question_note"
      ],
      "title": "CheatsheetQuestion",
      "type": "object"
    },
    "CheatsheetQuestionUID": {
      "properties": {
        "uniq

In [3]:
system_instruction = "You extract accurately data from a systematic review screening cheatsheet."

## OpenRouter (paid Gemma 3 27B IT with system instruction and response format); PDF passed as file

In [8]:
import requests
import base64

# First, encode and send the PDF
def encode_pdf_to_base64(pdf_path):
    with open(pdf_path, "rb") as pdf_file:
        return base64.b64encode(pdf_file.read()).decode('utf-8')

url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {os.getenv("OPENROUTER_API_KEY")}",
    "Content-Type": "application/json"
}

# Read and encode the PDF
cheatsheet_pdf_path = os.getenv("CHEATSHEET_PDF_PATH")
cheatsheet_base64_pdf = encode_pdf_to_base64(cheatsheet_pdf_path)
cheatsheet_data_url = f"data:application/pdf;base64,{cheatsheet_base64_pdf}"

# Read and encode the PDF
#diagram_pdf_path = os.getenv("CHEATSHEET_FLOW_DIAGRAM_PDF_PATH")
#diagram_base64_pdf = encode_pdf_to_base64(diagram_pdf_path)
#diagram_data_url = f"data:application/pdf;base64,{diagram_base64_pdf}"

# Initial request with the PDF
messages = [
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": system_instruction
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Here is the cheatsheet template:"
            },
            {
                "type": "file",
                "file": {
                    "filename": os.path.basename(cheatsheet_pdf_path),
                    "file_data": cheatsheet_data_url
                }
            },
        ]
    }
    #{
    #    "role": "user",
    #    "content": [
    #        {
    #            "type": "text",
    #            "text": "Here is the cheatsheet flow diagram:"
    #        },
    #        {
    #            "type": "file",
    #            "file": {
    #                "filename": os.path.basename(diagram_pdf_path),
    #                "file_data": diagram_data_url
    #            }
    #        },
    #    ]
    #}
]

payload = {
    "model": "google/gemma-3-27b-it",
    "messages": messages,
    "response_format": {
        "type": "json_schema",
        "json_schema": ScreeningCheatsheet.model_json_schema()
    }
}

response = requests.post(url, headers=headers, json=payload)
response_data = response.json()



In [9]:
response_data

{'id': 'gen-1752600270-lFQComnxOlayw48yrjRs',
 'provider': 'Novita',
 'model': 'google/gemma-3-27b-it',
 'object': 'chat.completion',
 'created': 1752600273,
 'choices': [{'logprobs': None,
   'finish_reason': 'stop',
   'native_finish_reason': 'stop',
   'index': 0,
   'message': {'role': 'assistant',
    'content': 'Okay, I understand. I will act as a data extractor, carefully reviewing information and providing responses based *solely* on the provided "shortages_eligibility_schema.pdf" cheatsheet. \n\n**Please provide me with the text or description of the study you want me to evaluate.** I need the study information to apply the criteria from the cheatsheet and give you the corresponding "YES", "NO", or "MAYBE" answers for each question.\n\nI will give my response in the following format:\n\n*   **Question 1: [YES/NO/MAYBE]**\n*   **Question 2: [YES/NO/MAYBE]**\n*   **Question 3: [YES/NO/MAYBE]**\n*   **Question 4: [YES/NO/MAYBE]**\n\n**Important:** I will *only* use the logic outl

## Sample: Ollama (Gemma 3 27B IT QAT Q4_0 with system instruction and response format); image passed as Base64

In [24]:
import ollama

# First, encode and send the PDF
def encode_to_base64(path):
    with open(path, "rb") as file:
        return base64.b64encode(file.read()).decode('utf-8')

# Read and encode the image
image_path = "/Volumes/home/anonymous/cheatsheet_parser/examples/title-abstract/media/cheatsheet-2.png"
base64_data = encode_to_base64(image_path)
#image_url = f"data:image/png;base64,{base64_data}"  # unsupported by ollama

response = ollama.chat(
	model="gemma3:27b-it-qat",
	messages=[
		{
			'role': 'user',
			'content': 'Describe this image:',
			'images': [base64_data]
		}
	]
)

response_data = response.model_dump_json()

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


In [26]:
from pprint import pprint
pprint(response_data)

('{"model":"gemma3:27b-it-qat","created_at":"2025-06-18T05:39:53.815499Z","done":true,"done_reason":"stop","total_duration":259466350000,"load_duration":12499311750,"prompt_eval_count":290,"prompt_eval_duration":96377542458,"eval_count":597,"eval_duration":150550183042,"message":{"role":"assistant","content":"Here\'s '
 'a description of the image you sent:\\n\\n**Overview**\\n\\nThe image shows '
 'a document titled \\"Appendix 7 – L1 Screening Form for Titles and '
 'Abstracts\\". It\'s a \\"Level 1 Cheat Sheet\\" (Version 3) designed to '
 'guide the initial screening of research studies related to \\"Gender Equity '
 '2\\".  It appears to be a part of a research project aimed at identifying '
 'interventions to optimize gender equity across various disciplines and '
 'implement them in academic health settings.\\n\\n**Key Elements**\\n\\n*   '
 '**Review Objective:** To identify existing interventions that optimize '
 'gender equity.\\n*   **Review Questions:**  The core question i

## Ollama (Gemma 3 27B IT QAT Q4_0 with system instruction and response format); images are passed as local paths

In [15]:
import ollama

cheatsheet_images = [
    "/Volumes/Users/Anonymous/Workshop/202406061326UTC-4_AI_ML_for_KS/OneDrive_LLM_Evidence_Synthesis/OneDrive_1_7-15-2025/scoping_shortages/cheatsheet-1.png",
    "/Volumes/Users/Anonymous/Workshop/202406061326UTC-4_AI_ML_for_KS/OneDrive_LLM_Evidence_Synthesis/OneDrive_1_7-15-2025/scoping_shortages/cheatsheet-2.png"
]

#flow_diagram_images = ["/Volumes/home/anonymous/cheatsheet_parser/examples/title-abstract/media/cheatsheet-flow-diagram.png"]

message = {
    "role": "user",
    "content": "Here is the cheatsheet template:",
    "images": cheatsheet_images
}

messages = [
    {
        "role": "system",
        "content": system_instruction
    },
    {
        "role": "user",
        "content": "Here is the cheatsheet template:",
        "images": cheatsheet_images
    }
    #{
    #    "role": "user",
    #    "content": "Here is the cheatsheet flow diagram:",
    #    "images": flow_diagram_images
    #}
]

response_with_schema = ollama.chat(
	model="gemma3:27b-it-qat",
	messages=messages,
    format=ScreeningCheatsheet.model_json_schema()  # https://ollama.com/blog/structured-outputs
)

response_with_schema_data = response_with_schema.model_dump_json()

INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


In [16]:
print(response_with_schema.message.content)

{"allowed_response_options": [], "questions": [
    {"question_uid": {
        "unique_question_id": 1
    }, "question_formulation": "Is this an observational study?", "responses": [
        {"value": {"string_value": "YES"
        , "unique_literal": "Yes"}, "notes": []
        }, {"value": {"string_value": "NO"
        , "unique_literal": "No"}, "notes": []
        }, {"value": {"string_value": "MAYBE"
        , "unique_literal": "Maybe"}, "notes": []
        }
    ],
    "question_note": "YES if the study:\n• Is a cohort study, case-control study, self-controlled study, or cross-sectional study\n• Is an interrupted time series analysis or controlled before and after study\nNO if the study:\n• Is a different type of observational study (e.g., case report, case series)\n• Is a randomized controlled trial\n• Is a literature review (e.g., narrative review, scoping review, systematic review, rapid review) or meta-analysis\n• Is a study protocol\n• Is a conference abstract\n• Is an opini

In [17]:
ScreeningCheatsheet.model_validate_json(response_with_schema.message.content)

ScreeningCheatsheet(allowed_response_options=[], questions=[CheatsheetQuestion(question_uid=CheatsheetQuestionUID(unique_question_id=1), question_formulation='Is this an observational study?', responses=[ResponseOption(value=ResponseOptionUniqueLiteral(string_value='YES', unique_literal='Yes'), notes=[]), ResponseOption(value=ResponseOptionUniqueLiteral(string_value='NO', unique_literal='No'), notes=[]), ResponseOption(value=ResponseOptionUniqueLiteral(string_value='MAYBE', unique_literal='Maybe'), notes=[])], question_note='YES if the study:\n• Is a cohort study, case-control study, self-controlled study, or cross-sectional study\n• Is an interrupted time series analysis or controlled before and after study\nNO if the study:\n• Is a different type of observational study (e.g., case report, case series)\n• Is a randomized controlled trial\n• Is a literature review (e.g., narrative review, scoping review, systematic review, rapid review) or meta-analysis\n• Is a study protocol\n• Is a c

In [ ]:
import yaml

data = {
    "response": {
        "content": ScreeningCheatsheet.model_validate_json(response_with_schema.message.content).model_dump()
    }
}

yaml_str = yaml.dump(data, default_flow_style=False, allow_unicode=True, indent=2)
print(yaml_str)

with open("untracked/shortages_eligibility_schema_2025-07-15.yml", "w") as f:
    yaml.dump(data, f, default_flow_style=False, allow_unicode=True, indent=2)

response:
  content:
    allowed_response_options: []
    flow_diagram: []
    flow_diagram_detected: false
    questions:
    - question_formulation: Is this an observational study?
      question_note: 'YES if the study:

        • Is a cohort study, case-control study, self-controlled study, or cross-sectional
        study

        • Is an interrupted time series analysis or controlled before and after study

        NO if the study:

        • Is a different type of observational study (e.g., case report, case series)

        • Is a randomized controlled trial

        • Is a literature review (e.g., narrative review, scoping review, systematic
        review, rapid review) or meta-analysis

        • Is a study protocol

        • Is a conference abstract

        • Is an opinion piece (e.g., editorial, commentary, perspective)

        • Is a review article or guideline'
      question_uid:
        unique_question_id: 1
      responses:
      - notes: []
        value:
        

## Gemini – agent wrapper from GERPA (outdated version); Gemma 3 27B IT free via Google API (no system instruction – passed in prompt; no response schema); PDF passed as a file

In [ ]:
# Create agent
LLMPROVIDER = "gemini"  # or "openrouter", "ollama"
llm_agent = agent(
    LLMPROVIDER,
    ScreeningCheatsheet,
    #system_instruction=system_instruction,
    model_name="gemma-3-27b-it"
    #model_name="gemma3:27b-it-qat"
    #model_name="google/gemma-3-27b-it:free"
    #model_name="meta-llama/llama-4-maverick:free"
)

# Test prompt
prompt = [
    system_instruction,
    "Here is the cheatsheet template:",
    f"{os.getenv("CHEATSHEET_PDF_PATH")}",  # note: docx is apparently unsupported
    #"Here is the cheatsheet flow diagram:",
    #f"{os.getenv("CHEATSHEET_FLOW_DIAGRAM_PDF_PATH")}"
]

start_time = time()
response = llm_agent(prompt)
end_time = time()
print(f"Execution time: {end_time - start_time:.4f} seconds")

INFO:httpx:HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
2025-06-18 01:09:08,307 - llm_agent_llm_provider - ERROR - Error generating response: OpenRouter API error: 'NoneType' object is not subscriptable
ERROR:llm_agent_llm_provider:Error generating response: OpenRouter API error: 'NoneType' object is not subscriptable


[{'role': 'user', 'content': [{'type': 'text', 'text': 'Here is the cheatsheet template:'}, {'type': 'text', 'text': 'Here is the cheatsheet flow diagram:'}, {'type': 'file', 'file': {'filename': 'cheatsheet.pdf', 'file_data': '...'}}, {'type': 'file', 'file': {'filename': 'cheatsheet-flow-diagram.pdf', 'file_data': '...'}}]}, {'role': 'system', 'content': "Respond with valid JSON matching this schema: {'$defs': {'CheatsheetQuestion': {'properties': {'question_uid': {'$ref': '#/$defs/CheatsheetQuestionUID'}, 'question_formulation': {'title': 'Question Formulation', 'type': 'string'}, 'responses': {'items': {'$ref': '#/$defs/ResponseOption'}, 'title': 'Responses', 'type': 'array'}, 'question_note': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'description': 'Any question-wide note(s), if present.', 'title': 'Question Note'}}, 'required': ['question_uid', 'question_formulation', 'responses', 'question_note'], 'title': 'CheatsheetQuestion', 'type': 'object'}, 'CheatsheetQuestionUID':

RuntimeError: OpenRouter API error: 'NoneType' object is not subscriptable